In [2]:
import logging
from oauth2client.client import flow_from_clientsecrets
from oauth2client.client import FlowExchangeError
from apiclient.discovery import build

SECRET_LOCATION = 'secret.json'
SCOPES = [
    'https://www.googleapis.com/auth/gmail.send',
    'https://www.googleapis.com/auth/userinfo.email',
    'https://www.googleapis.com/auth/userinfo.profile'
]

In [124]:
import httplib2
import base64
import json
from oauth2client.client import flow_from_clientsecrets
from apiclient.discovery import build
from email.mime.text import MIMEText

class Gmail:
    SECRET_LOCATION = 'secret.json'
    SCOPES = [
        'https://www.googleapis.com/auth/gmail.send',
        'https://www.googleapis.com/auth/userinfo.email',
        'https://www.googleapis.com/auth/userinfo.profile'
    ]
    def __init__(self):
        self.flow = flow_from_clientsecrets(self.SECRET_LOCATION,
                                       scope=' '.join(self.SCOPES),
                                       redirect_uri='urn:ietf:wg:oauth:2.0:oob')
    def get_autho_uri(self):
        autho_uri = self.flow.step1_get_authorize_url()
        return autho_uri
        
    def get_credentials(self, autho_code):
        credentials = self.flow.step2_exchange(autho_code)
        self.refresh_code = credentials.refresh_token
        return credentials
    
    def build_service(self, credentials):
        http = httplib2.Http()
        http = credentials.authorize(http)
        service = build('gmail', 'v1', http=http)
        return service
    
    def create_message(self, sender, to, subject, message_text):
        message = MIMEText(message_text)
        message['to'] = to
        message['from'] = sender
        message['subject'] = subject
        b64_bytes = base64.urlsafe_b64encode(message.as_bytes())
        b64_string = b64_bytes.decode()
        return {'raw' : b64_string}
    
    def send_message(self, service, user_id, message):
        message = (service.users().messages().send(userId=user_id, body=message).execute())
        print('Message Id: %s' % message['id'])
        return message
#         try:
#             
#         except errors.HttpError as error:
#             print('An error occurred: %s' %s)
        

In [125]:
gmail = Gmail()
print(gmail.get_autho_uri())

https://accounts.google.com/o/oauth2/auth?client_id=768255142362-4alnnus62lk9lnplf53th47t0eo59dr8.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.profile&access_type=offline&response_type=code


In [126]:
credentials = gmail.get_credentials('4/iQDv65cotGFKZew9KgEJfAJVGkqSxIqf2xDfMqqvBVVf5TsXugD9PO0')
service = gmail.build_service(credentials)
# print(credentials)

In [127]:
sender = '199511zc@gmail.com'
to = 'cz2465@columbia.edu'
subject = 'gmail api test'
message_text = 'This is a message sent from gmail api!!'
message = gmail.create_message(sender, to, subject, message_text)

In [128]:
gmail.send_message(service, '199511zc@gmail.com', message)

Message Id: 166d186c729d0989


{'id': '166d186c729d0989',
 'threadId': '166d186c729d0989',
 'labelIds': ['SENT']}